In [1]:
# Lung Cancer Detection Using Pretrained CNNs with Stratified Sampling and Class Balancing

# ❗ NOTE: This notebook requires PyTorch. If running in an environment without it, please install via pip:
# !pip install torch torchvision

# ✅ 1. Setup
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from PIL import Image
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.utils.class_weight import compute_class_weight

try:
    import torch
    import torch.nn as nn
    import torch.utils.checkpoint
    import torchvision.transforms as transforms
    from torch.utils.data import Dataset, DataLoader, Subset
    from torchvision import models
except ModuleNotFoundError:
    print("⚠️ PyTorch is not installed. Please run the following in a code cell:")
    print("!pip install torch torchvision")
    raise

#

In [5]:
# ✅ 2. Dataset Loader
class LungCancerClassificationDataset(Dataset):
    def __init__(self, root_dirs, class_names, transform=None):
        self.samples = []
        self.transform = transform
        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(class_names)}

        for class_name, path in zip(class_names, root_dirs):
            for img_name in os.listdir(path):
                img_path = os.path.join(path, img_name)
                if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.samples.append((img_path, self.class_to_idx[class_name]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("L")
        if self.transform:
            image = self.transform(image)
        return image, label


In [6]:

# ✅ 3. Evaluation Metrics
def evaluate_model(model, dataloader, device, num_classes):
    model.eval()
    y_true, y_pred, y_scores = [], [], []

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_scores.extend(probs.cpu().numpy())

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_scores = np.array(y_scores)
    y_true_bin = label_binarize(y_true, classes=list(range(num_classes)))

    acc = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true_bin, y_scores, multi_class='ovr')
    cm = confusion_matrix(y_true, y_pred)

    metrics = {
        "accuracy": acc,
        "auc": auc,
    }

    for i in range(num_classes):
        TP = cm[i, i]
        FN = cm[i, :].sum() - TP
        FP = cm[:, i].sum() - TP
        TN = cm.sum() - (TP + FP + FN)
        sensitivity = TP / (TP + FN + 1e-6)
        specificity = TN / (TN + FP + 1e-6)
        metrics[f"sensitivity_class_{i}"] = sensitivity
        metrics[f"specificity_class_{i}"] = specificity

    return metrics


In [7]:

# ✅ 4. GPU Memory Management
def free_gpu_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def enable_gradient_checkpointing(model, model_name):
    """
    Enable gradient checkpointing to reduce memory usage during training.
    This trades computation for memory by not storing all activations.
    """
    if model_name == "ResNet50":
        model.layer1.apply(lambda m: m.register_forward_hook(lambda _, __, ___, r: torch.utils.checkpoint.checkpoint(lambda x: x, r)))
        model.layer2.apply(lambda m: m.register_forward_hook(lambda _, __, ___, r: torch.utils.checkpoint.checkpoint(lambda x: x, r)))
        model.layer3.apply(lambda m: m.register_forward_hook(lambda _, __, ___, r: torch.utils.checkpoint.checkpoint(lambda x: x, r)))
        model.layer4.apply(lambda m: m.register_forward_hook(lambda _, __, ___, r: torch.utils.checkpoint.checkpoint(lambda x: x, r)))
    elif model_name == "DenseNet121":
        model.features.denseblock1.apply(lambda m: m.register_forward_hook(lambda _, __, ___, r: torch.utils.checkpoint.checkpoint(lambda x: x, r)))
        model.features.denseblock2.apply(lambda m: m.register_forward_hook(lambda _, __, ___, r: torch.utils.checkpoint.checkpoint(lambda x: x, r)))
        model.features.denseblock3.apply(lambda m: m.register_forward_hook(lambda _, __, ___, r: torch.utils.checkpoint.checkpoint(lambda x: x, r)))
        model.features.denseblock4.apply(lambda m: m.register_forward_hook(lambda _, __, ___, r: torch.utils.checkpoint.checkpoint(lambda x: x, r)))
    elif model_name == "EfficientNetB0":
        # Simplified for EfficientNet - you may need to adjust for specific blocks
        for block in model.features:
            if isinstance(block, nn.Sequential):
                block.apply(lambda m: m.register_forward_hook(lambda _, __, ___, r: torch.utils.checkpoint.checkpoint(lambda x: x, r)))
    elif model_name == "VGG19":
        # VGG is a simple sequential model, apply to groups of layers
        features_length = len(model.features)
        chunk_size = features_length // 4
        for i in range(0, features_length, chunk_size):
            for j in range(i, min(i + chunk_size, features_length)):
                if isinstance(model.features[j], nn.Conv2d):
                    model.features[j].register_forward_hook(lambda _, __, ___, r: torch.utils.checkpoint.checkpoint(lambda x: x, r))
    elif model_name == "InceptionV3":
        # For Inception, apply to main blocks
        model.Mixed_5b.register_forward_hook(lambda _, __, ___, r: torch.utils.checkpoint.checkpoint(lambda x: x, r))
        model.Mixed_5c.register_forward_hook(lambda _, __, ___, r: torch.utils.checkpoint.checkpoint(lambda x: x, r))
        model.Mixed_5d.register_forward_hook(lambda _, __, ___, r: torch.utils.checkpoint.checkpoint(lambda x: x, r))
        model.Mixed_6a.register_forward_hook(lambda _, __, ___, r: torch.utils.checkpoint.checkpoint(lambda x: x, r))
        model.Mixed_6b.register_forward_hook(lambda _, __, ___, r: torch.utils.checkpoint.checkpoint(lambda x: x, r))
        model.Mixed_6c.register_forward_hook(lambda _, __, ___, r: torch.utils.checkpoint.checkpoint(lambda x: x, r))
        model.Mixed_6d.register_forward_hook(lambda _, __, ___, r: torch.utils.checkpoint.checkpoint(lambda x: x, r))
        model.Mixed_6e.register_forward_hook(lambda _, __, ___, r: torch.utils.checkpoint.checkpoint(lambda x: x, r))
        model.Mixed_7a.register_forward_hook(lambda _, __, ___, r: torch.utils.checkpoint.checkpoint(lambda x: x, r))
        model.Mixed_7b.register_forward_hook(lambda _, __, ___, r: torch.utils.checkpoint.checkpoint(lambda x: x, r))
        model.Mixed_7c.register_forward_hook(lambda _, __, ___, r: torch.utils.checkpoint.checkpoint(lambda x: x, r))

def get_optimal_batch_size(model_name, available_memory_mb=4000):
    """
    Estimate optimal batch size based on model and available memory.
    This is a simplified estimation.
    """
    model_memory_requirements = {
        "ResNet50": 100,       # MB per sample
        "DenseNet121": 80,     # MB per sample
        "EfficientNetB0": 30,  # MB per sample
        "VGG19": 120,          # MB per sample
        "InceptionV3": 90      # MB per sample
    }

    # Default to lower batch size if model not in dictionary
    memory_per_sample = model_memory_requirements.get(model_name, 100)

    # Calculate batch size with a safety margin of 80%
    batch_size = int((available_memory_mb * 0.8) / memory_per_sample)

    # Set reasonable bounds
    batch_size = max(4, min(batch_size, 64))

    return batch_size


In [ ]:

# ✅ 5. Model Architecture Modifications
def get_model(model_name, num_classes=3):
    if model_name == "ResNet50":
        model = models.resnet50(pretrained=True)
        model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif model_name == "DenseNet121":
        model = models.densenet121(pretrained=True)
        model.features.conv0 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        model.classifier = nn.Linear(model.classifier.in_features, num_classes)
    elif model_name == "EfficientNetB0":
        model = models.efficientnet_b0(pretrained=True)
        model.features[0][0] = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    elif model_name == "VGG19":
        model = models.vgg19(pretrained=True)
        model.features[0] = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    elif model_name == "InceptionV3":
        model = models.inception_v3(pretrained=True, aux_logits=False)  # Disable auxiliary outputs
        model.Conv2d_1a_3x3.conv = nn.Conv2d(1, 32, kernel_size=3, stride=2)
        model.fc = nn.Linear(model.fc.in_features, num_classes)

    return model


In [ ]:

# ✅ 6. Training with Mixed Precision
def train_with_mixed_precision(model, train_loader, val_loader, criterion, optimizer, scheduler, device, patience=5, max_epochs=20):
    # Initialize scaler for mixed precision
    scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None
    best_val_loss = float('inf')
    counter = 0
    best_model = None

    for epoch in range(max_epochs):
        # Training phase
        model.train()
        train_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Use mixed precision if available
            if scaler is not None:
                with torch.cuda.amp.autocast():
                    outputs = model(images)
                    loss = criterion(outputs, labels)

                # Scale gradients and optimize
                optimizer.zero_grad()
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = model(images)
                loss = criterion(outputs, labels)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        # Validation phase - no need for mixed precision here
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        scheduler.step(avg_val_loss)

        print(f"Epoch {epoch+1}/{max_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Early stopping check
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            counter = 0
            best_model = model.state_dict().copy()
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping triggered after {epoch+1} epochs")
                break

    if best_model is not None:
        model.load_state_dict(best_model)

    return model


In [ ]:
google.colab import drive
drive.mount('content/drive')

In [ ]:

# ✅ 7. Define paths and transformations
benign_path = "<your_path>/Bengin cases"
malignant_path = "<your_path>/Malignant cases"
normal_path = "<your_path>/Normal cases"

class_names = ["Benign", "Malignant", "Normal"]
paths = [benign_path, malignant_path, normal_path]

# Enhanced transformations with data augmentation for medical images
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  # Horizontal flips are anatomically valid
    transforms.RandomRotation(10),  # Small rotations (10 degrees)
    transforms.RandomAffine(
        degrees=0,
        translate=(0.05, 0.05),  # Small translations
        scale=(0.95, 1.05),  # Subtle scaling
        fill=0  # Fill empty areas with black
    ),
    # Subtle brightness/contrast adjustments
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
])

# Keep validation/test transforms simple without augmentation
val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


In [ ]:

# ✅ 8. Visualize Class Distribution
full_dataset_for_counts = LungCancerClassificationDataset(paths, class_names, transform=None)
class_counts = {
    class_names[0]: len(os.listdir(benign_path)),
    class_names[1]: len(os.listdir(malignant_path)),
    class_names[2]: len(os.listdir(normal_path))
}

plt.figure(figsize=(8, 5))
plt.bar(class_counts.keys(), class_counts.values(), color=['skyblue', 'salmon', 'lightgreen'])
plt.title("Class Distribution in Lung Cancer Dataset")
plt.xlabel("Class")
plt.ylabel("Number of Images")
plt.tight_layout()
plt.savefig("class_distribution.png")
plt.show()


In [ ]:

# ✅ 9. Create datasets with appropriate transforms
train_dataset = LungCancerClassificationDataset(paths, class_names, transform=train_transform)
val_dataset = LungCancerClassificationDataset(paths, class_names, transform=val_test_transform)
test_dataset = LungCancerClassificationDataset(paths, class_names, transform=val_test_transform)

# ✅ 10. Stratified Split with the right transforms
labels = [label for _, label in train_dataset.samples]
indices = list(range(len(labels)))
train_idx, temp_idx = train_test_split(indices, test_size=0.3, stratify=labels, random_state=42)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, stratify=[labels[i] for i in temp_idx], random_state=42)

train_dataset = Subset(train_dataset, train_idx)
val_dataset = Subset(val_dataset, val_idx)
test_dataset = Subset(test_dataset, test_idx)

# ✅ 11. Create data loaders (with dynamic batch sizes implemented later)
default_batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=default_batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=default_batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=default_batch_size, shuffle=False)

# ✅ 12. Compute Class Weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float)

# ✅ 13. Define model architectures
architectures = {
    "ResNet50": models.resnet50,
    "DenseNet121": models.densenet121,
    "EfficientNetB0": models.efficientnet_b0,
    "VGG19": models.vgg19,
    "InceptionV3": models.inception_v3
}

# ✅ 14. Training Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
results = []

# Determine available GPU memory (simplified estimation)
if torch.cuda.is_available():
    # Get total GPU memory in MB
    total_memory = torch.cuda.get_device_properties(0).total_memory / (1024 * 1024)
    # Assume 75% of total memory is available
    available_memory = total_memory * 0.75
else:
    # Default value for CPU
    available_memory = 4000  # 4GB as default

for name in architectures.keys():
    print(f"\n{'='*50}")
    print(f"Training {name}...")
    print(f"{'='*50}")

    # Determine optimal batch size
    batch_size = get_optimal_batch_size(name, available_memory)
    print(f"Using batch size: {batch_size}")

    # Recreate data loaders with optimal batch size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Create model
    model = get_model(name)
    model = model.to(device)

    # Enable gradient checkpointing
    if torch.cuda.is_available():
        enable_gradient_checkpointing(model, name)

    # Training components
    criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

    # Train the model
    model = train_with_mixed_precision(
        model, train_loader, val_loader, criterion, optimizer, scheduler,
        device, patience=5, max_epochs=20
    )

    # Save the trained model
    model_path = f"{name}_lung_cancer_model.pth"
    torch.save(model.state_dict(), model_path)
    print(f"Model saved to {model_path}")

    # Evaluate on test set
    metrics = evaluate_model(model, test_loader, device, num_classes=3)
    metrics["model"] = name
    results.append(metrics)

    # Print metrics summary
    print(f"\nTest Metrics for {name}:")
    for key, value in metrics.items():
        if key != "model":
            print(f"  {key}: {value:.4f}")

    # Clean up to free memory
    del model, optimizer, scheduler, criterion
    free_gpu_memory()

    # Optional: Add a small delay to ensure memory is released
    time.sleep(2)

# ✅ 15. Save and Plot Results
df = pd.DataFrame(results)
df.to_csv("model_evaluation_results.csv", index=False)

# Create visualization of results
metric_names = ["accuracy", "auc"] + [m for m in df.columns if m.startswith("sensitivity") or m.startswith("specificity")]

plt.figure(figsize=(14, 10))
df.set_index("model")[metric_names].plot(kind='bar', figsize=(14, 7))
plt.title("Evaluation Metrics for CNN Models")
plt.ylabel("Score")
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig("model_metrics_comparison.png")
plt.show()

# ✅ 16. Display Confusion Matrices
for name in architectures.keys():
    print(f"\nLoading best model for {name}...")
    model = get_model(name)
    model.load_state_dict(torch.load(f"{name}_lung_cancer_model.pth"))
    model = model.to(device)

    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(torch.softmax(outputs, dim=1), dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(f"Confusion Matrix - {name}")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                     ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(f"{name}_confusion_matrix.png")
    plt.show()

    del model
    free_gpu_memory()

print("\nTraining and evaluation completed for all models.")
print(f"Results saved to 'model_evaluation_results.csv'")
print(f"Visualizations saved as PNG files.")

In [ ]:

# ✅ 9. Create datasets with appropriate transforms
train_dataset = LungCancerClassificationDataset(paths, class_names, transform=train_transform)
val_dataset = LungCancerClassificationDataset(paths, class_names, transform=val_test_transform)
test_dataset = LungCancerClassificationDataset(paths, class_names, transform=val_test_transform)



In [ ]:
# ✅ 10. Stratified Split with the right transforms
labels = [label for _, label in train_dataset.samples]
indices = list(range(len(labels)))
train_idx, temp_idx = train_test_split(indices, test_size=0.3, stratify=labels, random_state=42)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, stratify=[labels[i] for i in temp_idx], random_state=42)

train_dataset = Subset(train_dataset, train_idx)
val_dataset = Subset(val_dataset, val_idx)
test_dataset = Subset(test_dataset, test_idx)


In [ ]:

# ✅ 11. Create data loaders (with dynamic batch sizes implemented later)
default_batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=default_batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=default_batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=default_batch_size, shuffle=False)


In [ ]:

# ✅ 12. Compute Class Weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float)


In [ ]:

# ✅ 13. Define model architectures
architectures = {
    "ResNet50": models.resnet50,
    "DenseNet121": models.densenet121,
    "EfficientNetB0": models.efficientnet_b0,
    "VGG19": models.vgg19,
    "InceptionV3": models.inception_v3
}


In [ ]:

# ✅ 14. Training Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
results = []

# Determine available GPU memory (simplified estimation)
if torch.cuda.is_available():
    # Get total GPU memory in MB
    total_memory = torch.cuda.get_device_properties(0).total_memory / (1024 * 1024)
    # Assume 75% of total memory is available
    available_memory = total_memory * 0.75
else:
    # Default value for CPU
    available_memory = 4000  # 4GB as default

for name in architectures.keys():
    print(f"\n{'='*50}")
    print(f"Training {name}...")
    print(f"{'='*50}")

    # Determine optimal batch size
    batch_size = get_optimal_batch_size(name, available_memory)
    print(f"Using batch size: {batch_size}")

    # Recreate data loaders with optimal batch size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Create model
    model = get_model(name)
    model = model.to(device)

    # Enable gradient checkpointing
    if torch.cuda.is_available():
        enable_gradient_checkpointing(model, name)

    # Training components
    criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

    # Train the model
    model = train_with_mixed_precision(
        model, train_loader, val_loader, criterion, optimizer, scheduler,
        device, patience=5, max_epochs=20
    )

    # Save the trained model
    model_path = f"{name}_lung_cancer_model.pth"
    torch.save(model.state_dict(), model_path)
    print(f"Model saved to {model_path}")

    # Evaluate on test set
    metrics = evaluate_model(model, test_loader, device, num_classes=3)
    metrics["model"] = name
    results.append(metrics)

    # Print metrics summary
    print(f"\nTest Metrics for {name}:")
    for key, value in metrics.items():
        if key != "model":
            print(f"  {key}: {value:.4f}")

    # Clean up to free memory
    del model, optimizer, scheduler, criterion
    free_gpu_memory()

    # Optional: Add a small delay to ensure memory is released
    time.sleep(2)


In [ ]:

# ✅ 15. Save and Plot Results
df = pd.DataFrame(results)
df.to_csv("model_evaluation_results.csv", index=False)

# Create visualization of results
metric_names = ["accuracy", "auc"] + [m for m in df.columns if m.startswith("sensitivity") or m.startswith("specificity")]

plt.figure(figsize=(14, 10))
df.set_index("model")[metric_names].plot(kind='bar', figsize=(14, 7))
plt.title("Evaluation Metrics for CNN Models")
plt.ylabel("Score")
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig("model_metrics_comparison.png")
plt.show()


In [ ]:

# ✅ 16. Display Confusion Matrices
for name in architectures.keys():
    print(f"\nLoading best model for {name}...")
    model = get_model(name)
    model.load_state_dict(torch.load(f"{name}_lung_cancer_model.pth"))
    model = model.to(device)

    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(torch.softmax(outputs, dim=1), dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(f"Confusion Matrix - {name}")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                     ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(f"{name}_confusion_matrix.png")
    plt.show()

    del model
    free_gpu_memory()

print("\nTraining and evaluation completed for all models.")
print(f"Results saved to 'model_evaluation_results.csv'")
print(f"Visualizations saved as PNG files.")